# Importing necessary Modules

In [26]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Data Pre-Processing

In [27]:
dataset=os.path.join(os.path.abspath(''),r"dataset")

images=[]
classes=[]


In [28]:
for dirs in os.listdir(dataset):
    imagepath=os.path.join(dataset,dirs)
    for image in os.listdir(imagepath):
        img=cv2.imread(os.path.join(imagepath,image))
        resize=cv2.resize(img,(32,32))
        img=cv2.cvtColor(resize,cv2.COLOR_BGR2GRAY)
        img=cv2.equalizeHist(img)
        img=img/255
        images.append(img)
        classes.append(dirs)

In [29]:
len(images)

763

In [30]:
len(classes)

763

In [31]:
X=np.array(images)
Y=np.array(classes)


In [32]:
X.shape

(763, 32, 32)

In [33]:
Y.shape

(763,)

# Creating Hold out Environment

In [34]:
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.2)

In [35]:
X_train.shape

(610, 32, 32)

In [36]:
X_test.shape

(153, 32, 32)

In [37]:
Y_train.shape

(610,)

# Creating model

In [38]:
outputtobeclassified = len(os.listdir(dataset))

noOfFilters=60
sizeOfFilter1=(5,5)
sizeOfFilter2=(3,3)
sizeOfPool=(2,2)
noOfNode=500
model=Sequential()
model.add((Conv2D(noOfFilters, sizeOfFilter1, input_shape=(32,32,1), activation="relu")))
model.add((Conv2D(noOfFilters, sizeOfFilter1, activation="relu")))
model.add((MaxPooling2D(pool_size=sizeOfPool)))
model.add((Conv2D(noOfFilters//2, sizeOfFilter2, activation="relu")))
model.add((Conv2D(noOfFilters//2, sizeOfFilter2, activation="relu")))
model.add((MaxPooling2D(pool_size=sizeOfPool)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(noOfNode,activation="relu"))
model.add(Dropout(0.5)) 
model.add(Dense(outputtobeclassified,activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=['accuracy'])

In [40]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)

In [41]:
X_train.shape

(610, 32, 32, 1)

In [42]:
X_test.shape

(153, 32, 32, 1)

# One hot encoding 

In [46]:
y_train=to_categorical(Y_train, outputtobeclassified)
y_test=to_categorical(Y_test, outputtobeclassified)

# Image Augumentation

In [47]:
dataGen=ImageDataGenerator(width_shift_range=0.1,
                          height_shift_range=0.1,
                          zoom_range=0.2,
                          shear_range=0.1,
                          rotation_range=10)

# Training the  Model

In [51]:
model_training = model.fit_generator(dataGen.flow(X_train,y_train,batch_size=5),
                            steps_per_epoch=122,
                            epochs=122,
                            validation_data=(X_test,y_test),
                            shuffle=1)

Epoch 1/122
122/122 [==============================] - 5s 40ms/step - loss: 1.6109 - accuracy: 0.1951 - val_loss: 1.6059 - val_accuracy: 0.2026
Epoch 2/122
122/122 [==============================] - 5s 39ms/step - loss: 1.6078 - accuracy: 0.2164 - val_loss: 1.6077 - val_accuracy: 0.2353
Epoch 3/122
122/122 [==============================] - 5s 39ms/step - loss: 1.6044 - accuracy: 0.2246 - val_loss: 1.6016 - val_accuracy: 0.2549
Epoch 4/122
122/122 [==============================] - 5s 40ms/step - loss: 1.6038 - accuracy: 0.2180 - val_loss: 1.5950 - val_accuracy: 0.2418
Epoch 5/122
122/122 [==============================] - 5s 39ms/step - loss: 1.5873 - accuracy: 0.2361 - val_loss: 1.5883 - val_accuracy: 0.2418
Epoch 6/122
122/122 [==============================] - 5s 39ms/step - loss: 1.6032 - accuracy: 0.2344 - val_loss: 1.6008 - val_accuracy: 0.2418
Epoch 7/122
122/122 [==============================] - 5s 40ms/step - loss: 1.5828 - accuracy: 0.2459 - val_loss: 1.6604 - val_accuracy:

# Save Model

In [54]:
model.save('Celebrity_Prediction.h5')

# Validate the Model for new data

In [60]:
from tensorflow.keras.models import load_model
testmodel=load_model('Celebrity_Prediction.h5')

In [61]:
imOriginal=os.path.join(os.path.abspath(''),r"test\44.jpg")
img=cv2.imread(imOriginal)
resize=cv2.resize(img,(32,32)) # as done in previous file
img=cv2.cvtColor(resize,cv2.COLOR_BGR2GRAY)
img=cv2.equalizeHist(img)
img=img/255
img=np.array(img) # converting captured image into numpy array


In [62]:
img.shape

(32, 32)

In [63]:
img = np.reshape(img, (1, 32, 32, 1))

In [64]:
img.shape

(1, 32, 32, 1)

In [65]:
testmodel.predict(img)

array([[2.9930568e-04, 4.5058411e-01, 3.7017892e-04, 5.1193661e-06,
        5.4874128e-01]], dtype=float32)

In [66]:
predictedvalue=np.argmax(testmodel.predict(img), axis=1)

In [67]:
predictedvalue

array([4], dtype=int64)

In [116]:
# np.argmax  is used to find index of the class with highest probability

In [124]:
predictedvalue[0]

1

In [125]:
type(predictedvalue[0])

numpy.int64

In [69]:
mapping = {0:"ELON-MUSK", 1:"JIM-CARREY", 2:"MOHANLAL", 3:"MSDHONI", 4:"TOMCRUISE"}

In [71]:
if predictedvalue[0] in mapping:
    print(mapping[predictedvalue[0]])

TOMCRUISE
